### 3. Programación No Lineal con Ipopt

La programación no lineal es necesaria para problemas como el flujo AC, donde las ecuaciones incluyen términos no lineales (e.g., ( $\cos(\theta)$ )).

#### Ejemplo: Flujo AC Simplificado

Modelaremos un sistema pequeño con restricciones no lineales para flujos de potencia.

### Variables de decisión

* $PGEN_i \geq 0$ : potencia activa generada en el nodo $i$ (MW)
* $QGEN_i \geq 0$ : potencia reactiva generada en el nodo $i$ (MVAR)
* $\theta_i $ : ángulo de tensión en el nodo $i$ (rad)
* $V_i $ : tensión en el nodo $i$ (kV)
* $P_{km}$ : : flujo de potencia activa en la línea que va del nodo $k$ al nodo $m$ (MW)

--- 

### Función objetivo
La función objetivo es minimizar la potencia activa generada en el sistema, mientras se garantiza la estabilidad y seguridad del sistema. La función objetivo se puede expresar como:
$$ \min \sum_{i \in \mathcal{N}} C_i P_{GEN_i}$$

Donde $C_i$ es el costo de generación por unidad de potencia $/MW activa en el nodo $i$.

---

### Restricciones

1. **Nodo Slack**:

$$\theta_{1} = 0.0, V_1 = 1.0$$

2. **Limites de voltaje**:

$$0.95 \leq V_i \leq 1.05, \forall i$$

3. **Capacidad de generación**:

$$0 \leq P_{GEN_i} \leq P_{MAX_i}, \forall i$$
$$0 \leq Q_{GEN_i} \leq Q_{MAX_i}, \forall i$$

4. **Ecuación de flujo en cada línea (no lineal)**

$$ P_{km} = g_{km} V_k^2 - V_k V_m (g_{km} \cos{(\theta_k - \theta_m)} + b_{km} \sin{(\theta_k - \theta_m)}) $$

5. **Ecuación de balance de potencia activa en cada nodo**:

$$ P_{GEN_i} - P_{load} = \sum_{j:FROM=i}{P_{ij}} + \sum_{j:TO=j}{P_{ji}}$$


In [20]:
using JuMP
using Ipopt
using DataFrames

# Datos (nodos y líneas)
nodes = DataFrame(
    Nodo = [1, 2],
    PGEN = [100.0, 150.0],  # Potencia generada en MW (Limite de generación)
    QGEN = [50.0, 75.0],    # Potencia reactiva generada en MVAR (Limite de generación)
    PLOAD = [50.0, 60.0],   # Potencia demandada en MW
    QLOAD = [20.0, 30.0],    # Potencia reactiva demandada en MVAR
    COSTO = [10.0, 15.0],  # Costo de generación por MW
)

lines = DataFrame(
    FROM = [1],     # Nodo de origen
    TO = [2],       # Nodo de destino
    R = [0.01],     # Resistencia de la línea en pu
    X = [0.05],     # Reactancia de la línea en pu
    LIM1 = [100.0]  # Límite de potencia activa en MW
)

num_lines = nrow(lines)
num_nodes = nrow(nodes)

# Definiendo el modelo
modelo = Model(Ipopt.Optimizer)
set_optimizer_attribute(modelo, "print_level", 0)  # Suprime la salida de Ipopt
set_optimizer_attribute(modelo, "tol", 1e-4)  # Tolerancia de convergencia

# Variables de decisión
@variable(modelo, PGEN[1:num_nodes]  >= 0)  # Potencia activa en MW
@variable(modelo, QGEN[1:num_nodes]  >= 0)  # Potencia reactiva en MVAR
@variable(modelo, θ[1:num_nodes])  # Ángulo de voltaje en radianes
@variable(modelo, V[1:num_nodes])  # Voltaje en pu
@variable(modelo, Pkm[1:num_lines])  # Potencia activa en MW de las líneas
@variable(modelo, Pmk[1:num_lines])  # Potencia activa en MW de las líneas

# Función objetivo: minimizar el costo total de generación
@objective(modelo, Min, sum(nodes.COSTO .* PGEN))

# Restricciones nodo slack
@constraint(modelo, V[1] == 1.0)  # Nodo slack (nodo 1) con voltaje fijo en 1.0 pu
@constraint(modelo, θ[1] == 0.0)  # Nodo slack (nodo 1) con ángulo fijo en 0 radianes

# Restricciones de niveles de tensión
@constraint(modelo, V[1:num_nodes] .>= 0.95)  # Límite inferior de voltaje
@constraint(modelo, V[1:num_nodes] .<= 1.05)  # Límite superior de voltaje

# Restricciones de capacidad de generación
@constraint(modelo, 0 .<= PGEN .<= nodes.PGEN)  # Límites de generación activa
@constraint(modelo, 0 .<= QGEN .<= nodes.QGEN)  # Límites de generación reactiva

# Restricciones de las líneas
for i in 1:num_lines
    k = lines.FROM[i]
    m = lines.TO[i]
    gkm = real(1/((lines.R[i]) + 1im * lines.X[i]))  # admitancia de la línea
    bkm = imag(1/((lines.R[i]) + 1im * lines.X[i]))  # admitancia de la línea

    # Restricciones de flujo de potencia activa
    @NLconstraint(modelo, Pkm[i] == (V[k])^2 * gkm - V[k] * V[m] * (gkm * cos(θ[k] - θ[m]) + bkm * sin(θ[k] - θ[m])))
    @NLconstraint(modelo, Pmk[i] == (V[m])^2 * gkm - V[m] * V[k] * (gkm * cos(θ[m] - θ[k]) + bkm * sin(θ[m] - θ[k])))
end

# Balance de potencia activa en cada nodo
for i in 1:num_nodes
    Pki = 0
    Pik = 0
    for j in 1:num_lines
        if lines.FROM[j] == i
            Pki += Pkm[j]
        elseif lines.TO[j] == i
            Pik += Pmk[j]
        end
    end
    @constraint(modelo, PGEN[i] - nodes.PLOAD[i] == Pki - Pik)  # Balance de potencia activa
end

optimize!(modelo)  # Resolver el modelo

# Resultados
if termination_status(modelo) == MOI.OPTIMAL
    println("Costo óptimo: ", objective_value(modelo))
    println("Generación activa: ", value.(pG))
    println("Voltajes: ", value.(Vi))
else
    println("No se encontró solución óptima: ", termination_status(modelo))
    println(modelo)
end

No se encontró solución óptima: LOCALLY_SOLVED
Min 10 PGEN[1] + 15 PGEN[2]
Subject to
 V[1] == 1
 θ[1] == 0
 PGEN[1] - Pkm[1] == 50
 PGEN[2] + Pmk[1] == 60
 V[1] >= 0.95
 V[2] >= 0.95
 V[1] <= 1.05
 V[2] <= 1.05
 PGEN[1] in [0, 100]
 PGEN[2] in [0, 150]
 QGEN[1] in [0, 50]
 QGEN[2] in [0, 75]
 PGEN[1] >= 0
 PGEN[2] >= 0
 QGEN[1] >= 0
 QGEN[2] >= 0
 (Pkm[1] - (V[1] ^ 2.0 * 3.846153846153846 - V[1] * V[2] * (3.846153846153846 * cos(θ[1] - θ[2]) + -19.23076923076923 * sin(θ[1] - θ[2])))) - 0.0 == 0
 (Pmk[1] - (V[2] ^ 2.0 * 3.846153846153846 - V[2] * V[1] * (3.846153846153846 * cos(θ[2] - θ[1]) + -19.23076923076923 * sin(θ[2] - θ[1])))) - 0.0 == 0

